This is a Notebook to test the ADAM batch OPM API.<br>
It was taken from test.py

In [1]:
from adam import Batch

In [2]:
import time

In [3]:
state_vec = [130347560.13690618,
             -74407287.6018632,
             -35247598.541470632,
             23.935241263310683,
             27.146279819258538,
             10.346605942591514]

In [91]:
batch_run = Batch()
batch_run.set_start_time('2017-11-01T00:00:00Z')
batch_run.set_end_time('2117-11-05T00:00:00Z')
batch_run.set_state_vector('2017-11-01T00:00:00.000Z', state_vec)

#TODO
# batch_run.originator('AstrogatorJohn')

In [92]:
print("Initial state %s" % batch_run)
print
print(batch_run.generate_opm())

Initial state Batch None, None
CCSDS_OPM_VERS = 2.0
CREATION_DATE = 2017-10-28 04:48:38.357582
ORIGINATOR = Tatiana
COMMENT Cartesian coordinate system
OBJECT_NAME = dummy
OBJECT_ID = 001
CENTER_NAME = SUN
REF_FRAME = ITRF-97
TIME_SYSTEM = UTC
EPOCH = 2017-11-01T00:00:00.000Z
X = 130347560.13690618
Y = -74407287.6018632
Z = -35247598.54147063
X_DOT = 23.935241263310683
Y_DOT = 27.146279819258538
Z_DOT = 10.346605942591514
MASS = 1000
SOLAR_RAD_AREA = 20
SOLAR_RAD_COEFF = 1
DRAG_AREA = 20
DRAG_COEFF = 2.2


In [93]:
batch_run.submit()
print
print("Final state %s" % batch_run)

#while not batch_run.is_ready():
#    time.sleep(5)

Final state Batch 7584c6b0-0b77-4142-a423-1414ad2f5884, PENDING


In [94]:
batch_run.is_ready()

False

In [95]:
batch_run.get_calc_state()

'PENDING'

In [77]:
batch_run._load_state()

'COMPLETED'

In [78]:
part_count = batch_run.get_parts_count()
print("Final state %s, part count %s" % (batch_run.get_calc_state(), part_count))

Final state COMPLETED, part count 1


In [79]:
eph = batch_run.get_part_ephemeris(1)
print("Ephemeris")
print(eph)

Ephemeris
stk.v.9.0

# WrittenBy    STK_Components_2017 r2(17.2.389.0)

BEGIN Ephemeris

NumberOfEphemerisPoints	18263
ScenarioEpoch	1 Nov 2017 00:00:00
InterpolationMethod	Hermite
InterpolationSamplesM1	2
CentralBody	Sun
CoordinateSystem	ICRF

EphemerisTimePosVel

0 130347560136.90617 -74407287601.8632 -35247598541.470634 23935.241263310683 27146.27981925854 10346.605942591514
86400 132397901125.11124 -72051923854.69356 -34348940387.921593 23525.854245647355 27374.491804266334 10455.050165711476
172800 134412746598.11105 -69677241472.5663 -33441072670.539497 23113.649317600673 27593.466375824886 10559.786382234655
259200 136391869491.7789 -67284039369.77271 -32524316668.11368 22699.00550508213 27803.18788051917 10660.794817961909
345600 138335075297.39645 -64873117067.26436 -31598995053.79208 22282.29719703383 28003.657840140942 10758.062940190648
432000 140242201614.67502 -62445273232.61813 -30665431277.63819 21863.893084085983 28194.894387656073 10851.58526256211
518400 142113117616

https://pro-equinox-162418.appspot.com/_ah/api/adam/v1/batch/7584c6b0-0b77-4142-a423-1414ad2f5884